In [0]:
!pip install tensorflow-gpu==1.15.0 #install tensorflow

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

!git clone https://github.com/tensorflow/models.git

%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

!python object_detection/builders/model_builder_test.py  #test object detection model

In [0]:
!pip install imutils
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [0]:

#to convert to csv format from file names

import os
from PIL import Image
import csv
imagePaths1 = '/content/ProductImagesFromShelves'
imagePaths2 = '/content/ShelfImages/train' # or /test
csv_list=[]

for b in 0,1,2,3,4,5,6,7,8,9,10:
  for file in os.listdir(f"{imagePaths1}/{b}"):	
    try:
      x=file.split(os.path.sep)[-1]
      x = x.split(".")[-3]
      x=x+'.JPG'
      img = Image.open(f"{imagePaths2}/{x}")
      width, height = img.size
      y = file.split(".")[-2]
      y = np.array(y.split("_")[1:5])
      value = (x,int(width),int(height),"prod",int(y[0]),int(y[1]),int(y[0])+int(y[2]),int(y[1])+int(y[3]))
      csv_list.append(value)
    except:
      continue
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
csv_df = pd.DataFrame(csv_list, columns=column_name)
csv_df.to_csv('train.csv', index=None)  #or test.csv
print('Successfully converted to csv.')


In [0]:
%cd /content/models/research/
!python setup.py install

In [0]:
%cd /content/models/research

!protoc object_detection/protos/*.proto --python_out=.

%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [0]:
%cd object_detection/
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
#to generate tfrecord from csv

%cd /content/drive/My\ Drive/Grocery/OD  #path to tf_record.py
!python generate_tfrecord.py --csv_input=data/test.csv  --output_path=data/test.record  --image_dir=images/test

In [0]:
!wget https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/ssd_mobilenet_v1_coco.config

In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz

In [0]:
!tar -xvzf ssd_mobilenet_v1_coco_2018_01_28.tar.gz

In [0]:
%cd /content/
%cp -av "/content/drive/My Drive/Grocery/images" "/content/models/research/object_detection/" 
%cp -av "/content/drive/My Drive/Grocery/test.record" "/content/models/research/object_detection/data/" 
%cp -av "/content/drive/My Drive/Grocery/train.record" "/content/models/research/object_detection/data/"
%cp -av "/content/drive/My Drive/Grocery/training" "/content/models/research/object_detection/"
%cp -av "/content/drive/My Drive/Grocery/ssd_mobilenet_v1_coco_2018_01_28" "/content/models/research/object_detection/"   

In [0]:
%cd /content/models/research/object_detection/

In [0]:
!python legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config

In [0]:
%cp -av "/content/models/research/object_detection/training" "/content/drive/My Drive/training" 

In [0]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-11018 --output_directory prod_graph

In [0]:
%cp -av "/content/models/research/object_detection/prod_graph" "/content/drive/My Drive/Grocery" 